In [26]:
import httpx
import json
from typing import Any
import os

import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("Error: API Key not found!")
else:
    print(f"API Key found")

genai.configure(api_key=api_key)

API Key found


In [27]:
async def get_coordinates(city: str) -> dict[str, Any]:
    """
    Get latitude and longitude for a city using Open-Meteo's geocoding API.
    """
    # Clean up city name - remove state abbreviations like ", IN" or ", CA"
    city_clean = city.split(",")[0].strip()
    
    url = "https://geocoding-api.open-meteo.com/v1/search"
    params = {"name": city_clean, "count": 1, "format": "json"}
    
    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params)
        response.raise_for_status()
        data = response.json()
    
    if not data.get("results"):
        raise ValueError(f"City not found: {city}")
    
    result = data["results"][0]
    return {
        "name": result["name"],
        "country": result.get("country", "Unknown"),
        "latitude": result["latitude"],
        "longitude": result["longitude"]
    }

In [28]:
# Weather code descriptions from Open-Meteo documentation
# Weather codes generated using Claude Sonnet 4.5
WEATHER_CODES = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Light drizzle",
    53: "Moderate drizzle",
    55: "Dense drizzle",
    61: "Slight rain",
    63: "Moderate rain",
    65: "Heavy rain",
    71: "Slight snow",
    73: "Moderate snow",
    75: "Heavy snow",
    77: "Snow grains",
    80: "Slight rain showers",
    81: "Moderate rain showers",
    82: "Violent rain showers",
    85: "Slight snow showers",
    86: "Heavy snow showers",
    95: "Thunderstorm",
    96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail",
}


async def get_current_weather(city: str) -> dict[str, Any]:
    """
    Get current weather conditions for a city.
    
    Args:
        city: Name of the city (e.g., "Boston", "London", "Tokyo")
    
    Returns:
        Dictionary with current weather data including temperature,
        humidity, wind speed, and conditions.
    """
    # Get coordinates for the city
    location = await get_coordinates(city)
    
    # Fetch current weather from Open-Meteo
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": location["latitude"],
        "longitude": location["longitude"],
        "current": [
            "temperature_2m",
            "relative_humidity_2m",
            "apparent_temperature",
            "weather_code",
            "wind_speed_10m",
            "wind_direction_10m",
        ],
        "temperature_unit": "fahrenheit",
        "wind_speed_unit": "mph",
    }
    
    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params)
        response.raise_for_status()
        data = response.json()
    
    current = data["current"]
    weather_code = current["weather_code"]
    
    return {
        "location": f"{location['name']}, {location['country']}",
        "temperature_f": current["temperature_2m"],
        "feels_like_f": current["apparent_temperature"],
        "humidity_percent": current["relative_humidity_2m"],
        "wind_speed_mph": current["wind_speed_10m"],
        "wind_direction_deg": current["wind_direction_10m"],
        "conditions": WEATHER_CODES.get(weather_code, "Unknown"),
        "weather_code": weather_code,
    }

In [29]:
async def get_weather_forecast(city: str, days: int = 5) -> dict[str, Any]:
    """
    Get weather forecast for a city.
    
    Args:
        city: Name of the city (e.g., "Boston", "London", "Tokyo")
        days: Number of days to forecast (1-7, default 5)
    
    Returns:
        Dictionary with daily forecast data including high/low temps,
        precipitation probability, and conditions.
    """
    # Validate days parameter and ensure it's an integer
    days = int(max(1, min(7, days)))
    
    # Get coordinates for the city
    location = await get_coordinates(city)
    
    # Fetch forecast from Open-Meteo
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": location["latitude"],
        "longitude": location["longitude"],
        "daily": [
            "temperature_2m_max",
            "temperature_2m_min",
            "precipitation_probability_max",
            "weather_code",
            "wind_speed_10m_max",
        ],
        "temperature_unit": "fahrenheit",
        "wind_speed_unit": "mph",
        "forecast_days": days,
    }
    
    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params)
        response.raise_for_status()
        data = response.json()
    
    daily = data["daily"]
    
    # Build forecast list
    forecast = []
    for i in range(len(daily["time"])):
        weather_code = daily["weather_code"][i]
        forecast.append({
            "date": daily["time"][i],
            "high_f": daily["temperature_2m_max"][i],
            "low_f": daily["temperature_2m_min"][i],
            "precipitation_chance_percent": daily["precipitation_probability_max"][i],
            "max_wind_mph": daily["wind_speed_10m_max"][i],
            "conditions": WEATHER_CODES.get(weather_code, "Unknown"),
        })
    
    return {
        "location": f"{location['name']}, {location['country']}",
        "forecast_days": days,
        "daily_forecast": forecast,
    }

In [30]:
import google.generativeai as genai

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
genai.configure(api_key=GOOGLE_API_KEY)

weather_tools = [{
    "function_declarations": [
        {
            "name": "get_current_weather",
            "description": "Get current weather conditions for a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City name (e.g., 'Boston')"}
                },
                "required": ["city"]
            }
        },
        {
            "name": "get_weather_forecast",
            "description": "Get weather forecast for a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City name (e.g., 'Boston')"},
                    "days": {"type": "integer", "description": "Days to forecast (1-7)"}
                },
                "required": ["city"]
            }
        }
    ]
}]

print("Gemini configured with weather tools")

Gemini configured with weather tools


In [31]:
async def execute_tool(tool_name: str, args: dict) -> str:
    """
    Execute a weather tool and return the result.
    This simulates what the MCP server would do.
    """
    if tool_name == "get_current_weather":
        result = await get_current_weather(args["city"])
        return json.dumps(result, indent=2)
    elif tool_name == "get_weather_forecast":
        days = int(args.get("days", 5))  # Convert to int - Gemini sends floats
        result = await get_weather_forecast(args["city"], days)
        return json.dumps(result, indent=2)
    else:
        return json.dumps({"error": f"Unknown tool: {tool_name}"})

In [32]:
async def chat_with_weather(user_prompt: str) -> str:
    """
    Send a prompt to Gemini and handle any tool calls.
    """
    model = genai.GenerativeModel(
        model_name="gemini-flash-latest",
        tools=weather_tools
    )
    
    chat = model.start_chat()
    
    print(f"User: {user_prompt}\n")
    
    # Send initial message
    response = chat.send_message(user_prompt)
    
    # Keep processing until we get a text response
    max_iterations = 10  # Safety limit
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        
        # Check if response has any parts
        if not response.candidates or not response.candidates[0].content.parts:
            break
            
        # Check if response has any function calls
        has_function_call = False
        for part in response.candidates[0].content.parts:
            if hasattr(part, 'function_call') and part.function_call and part.function_call.name:
                has_function_call = True
                func_call = part.function_call
                tool_name = func_call.name
                args = dict(func_call.args)
                
                print(f"Gemini called tool: {tool_name}")
                print(f"Arguments: {args}\n")
                
                # Execute the tool
                result = await execute_tool(tool_name, args)
                
                print(f"Tool result:\n{result}\n")
                
                # Send the result back to Gemini
                response = chat.send_message(
                    genai.protos.Content(
                        parts=[genai.protos.Part(
                            function_response=genai.protos.FunctionResponse(
                                name=tool_name,
                                response={"result": result}
                            )
                        )]
                    )
                )
                break  # Process one function call at a time
        
        if not has_function_call:
            # No more function calls, we have the final response
            break
    
    # Safely extract text from response
    try:
        final_response = response.text
    except ValueError:
        # If .text fails, try to extract text from parts manually
        texts = []
        if response.candidates and response.candidates[0].content.parts:
            for part in response.candidates[0].content.parts:
                if hasattr(part, 'text') and part.text:
                    texts.append(part.text)
        final_response = "\n".join(texts) if texts else "[No text response from model]"
    
    print(f"Gemini: {final_response}")
    return final_response

In [33]:
# Test 1: Current weather query
print("=" * 60)
print("TEST 1: Current Weather Query")
print("=" * 60)
await chat_with_weather("What's the current weather like in Indianapolis, IN?")

TEST 1: Current Weather Query
User: What's the current weather like in Indianapolis, IN?

Gemini called tool: get_current_weather
Arguments: {'city': 'Indianapolis, IN'}

Tool result:
{
  "location": "Indianapolis, United States",
  "temperature_f": 38.1,
  "feels_like_f": 32.5,
  "humidity_percent": 50,
  "wind_speed_mph": 0.9,
  "wind_direction_deg": 194,
  "conditions": "Overcast",
  "weather_code": 3
}

Gemini: The current weather in Indianapolis, IN is overcast with a temperature of 38.1°F, but it feels like 32.5°F. The humidity is 50%, and the wind speed is 0.9 mph.


'The current weather in Indianapolis, IN is overcast with a temperature of 38.1°F, but it feels like 32.5°F. The humidity is 50%, and the wind speed is 0.9 mph.'

In [34]:
import google.generativeai as genai

for model in genai.list_models():
    if "generateContent" in model.supported_generation_methods:
        print(model.name)

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-prev

In [35]:
# Test 2: Forecast query
print("\n" + "=" * 60)
print("TEST 2: Weather Forecast Query")
print("=" * 60)
await chat_with_weather("What will the weather be like in Indianapolis for the next 5 days?")


TEST 2: Weather Forecast Query
User: What will the weather be like in Indianapolis for the next 5 days?

Gemini called tool: get_weather_forecast
Arguments: {'city': 'Indianapolis', 'days': 5.0}

Tool result:
{
  "location": "Indianapolis, United States",
  "forecast_days": 5,
  "daily_forecast": [
    {
      "date": "2026-01-04",
      "high_f": 38.3,
      "low_f": 24.8,
      "precipitation_chance_percent": 0,
      "max_wind_mph": 6.0,
      "conditions": "Overcast"
    },
    {
      "date": "2026-01-05",
      "high_f": 49.6,
      "low_f": 30.8,
      "precipitation_chance_percent": 1,
      "max_wind_mph": 10.8,
      "conditions": "Overcast"
    },
    {
      "date": "2026-01-06",
      "high_f": 55.4,
      "low_f": 37.1,
      "precipitation_chance_percent": 13,
      "max_wind_mph": 13.7,
      "conditions": "Light drizzle"
    },
    {
      "date": "2026-01-07",
      "high_f": 52.9,
      "low_f": 39.7,
      "precipitation_chance_percent": 5,
      "max_wind_mph": 9.

'The weather forecast for Indianapolis for the next 5 days is as follows:\n\n*   **Day 1 (2026-01-04):** Overcast, with a high of 38.3°F and a low of 24.8°F. Max wind will be 6.0 mph.\n*   **Day 2 (2026-01-05):** Overcast, with a high of 49.6°F and a low of 30.8°F. Max wind will be 10.8 mph.\n*   **Day 3 (2026-01-06):** Light drizzle, with a high of 55.4°F and a low of 37.1°F. There is a 13% chance of precipitation and max wind will be 13.7 mph.\n*   **Day 4 (2026-01-07):** Clear sky, with a high of 52.9°F and a low of 39.7°F. Max wind will be 9.0 mph.\n*   **Day 5 (2026-01-08):** Overcast, with a high of 54.2°F and a low of 41.4°F. There is a 58% chance of precipitation and max wind will be 7.2 mph.'